# Cell2Cell Group Assignment
*Class*: Marketing Analytics (EWMBA263 -1)  
*Team*: Albert Deng, Peter Pang, Alex Im, and DJ Prahladka

## 0. Data Initialization

In [11]:
import pandas
import shared.mba263 as mba263
import matplotlib.pyplot as plt
import numpy as np

data = pandas.read_csv('shared/cell2cell.csv')
data.head(10)

,customer,calibrat,churn,churndep,revenue,mou,recchrge,directas,overage,roam,...,retaccpt,newcelly,newcelln,refer,incmiss,income,mcycle,setprcm,setprc,retcall
0,1000002,0,0,NaN,57.492500,482.75,37.424999,0.2475,22.75,0.000000,...,0,0,1,0,0,5,0,0,149.989990,0
1,1000006,0,0,NaN,82.275002,1312.25,75.000000,1.2375,0.00,0.000000,...,0,1,0,0,0,6,0,0,9.989998,0
2,1000010,0,0,NaN,31.662500,25.50,29.990000,0.2475,0.00,0.000000,...,0,0,1,0,0,9,0,0,29.989990,0
3,1000011,0,0,NaN,62.127499,97.50,65.985001,2.4750,0.00,0.000000,...,0,1,0,0,0,6,0,0,29.989990,0
4,1000014,0,0,NaN,25.225000,2.50,25.000000,0.0000,0.00,0.000000,...,0,1,0,0,0,7,0,0,29.989990,0
5,1000015,0,0,NaN,212.515000,1971.50,84.989998,2.2275,249.50,35.497501,...,0,1,0,0,0,3,0,0,79.989990,0
6,1000016,0,0,NaN,42.564999,270.50,37.480000,0.2475,6.00,0.000000,...,0,0,1,0,0,1,0,0,29.989990,0
7,1000018,0,0,NaN,35.590000,153.00,29.990000,0.0000,16.00,0.000000,...,0,0,1,0,0,4,0,0,199.989990,0
8,1000019,0,0,NaN,55.270000,1212.75,49.990002,0.7425,0.00,1.285000,...,0,0,1,0,0,3,0,0,9.989998,0
9,1000020,0,0,NaN,50.965000,162.00,69.989998,0.0000,2.50,0.000000,...,0,1,0,0,0,1,0,0,9.989998,0


## 1. Data Selection and Analysis
### a) Variable Definition

In [12]:
def classify_variables(variables):
    continuous = []
    decile = []
    indicator = []

    for variable in variables:
        if data[variable].max() == 1 and data[variable].min() == 0:
            indicator.append(variable)
        elif data[variable].max() == 9 and data[variable].min() == 0:
            decile.append(variable)
        else:
            continuous.append(variable)
    return continuous, decile, indicator

all_variables = ['revenue','mou','recchrge','directas','overage','roam','changem','changer','dropvce','blckvce','unansvce','custcare','threeway','mourec','outcalls','incalls','peakvce','opeakvce','dropblk','callfwdv','callwait','months','uniqsubs','actvsubs','phones','models','eqpdays','age1','age2','children','credita','creditaa','prizmrur','prizmub','prizmtwn','refurb','webcap','truck','rv','occprof','occcler','occcrft','occstud','occhmkr','occret','occself','ownrent','marryun','marryyes','mailord','mailres','mailflag','travel','pcown','creditcd','retcalls','retaccpt','newcelly','newcelln','refer','incmiss','income','mcycle','setprcm','setprc','retcall']

# Classify variable types
continuous_variables, decile_variables, indicator_variables = classify_variables(all_variables)

### b) Exploratory Data Analysis
Define functions to classify whether a variable has a significant difference in predicting churn

In [13]:
# Define a function that checks whether the distribution of a variable is significantly different between two groups
def is_indicator_significant(dataset, predict, variable, threshold):
    chi2, p = mba263.chi2(dataset[predict], dataset[variable])
    # Return whether the p-value is below the threshold
    return p < threshold

def sort_indicators_by_p(dataset, predict, variables):
    p_values = []
    for variable in variables:
        p = mba263.chi2(dataset[predict], dataset[variable])[1]
        p_values.append(p)
    return pandas.DataFrame({'variable': variables, 'p': p_values}).sort_values('p')['variable'].values

In [14]:
test = data.loc[data['calibrat'] == 1]

# Filter the indicator_variables list to only include significant variables
sig_indicator_variables = [variable for variable in indicator_variables 
                           if is_indicator_significant(test, 'churndep', variable, 0.03)]
sig_indicator_variables = sort_indicators_by_p(test, 'churndep', sig_indicator_variables)
print(sig_indicator_variables)

['retcall' 'webcap' 'creditaa' 'setprcm' 'refurb' 'mailres' 'mailord'
 'marryun' 'credita' 'incmiss' 'ownrent' 'creditcd' 'newcelly' 'prizmtwn']


In [15]:
test.groupby('newcelly')['churn'].mean()

newcelly
0    0.503315
1    0.486147
Name: churn, dtype: float64

### c) Variable Selection

In [16]:
# Append sig_indicator_variables, decile_variables, and continuous variables together
selected_variables = np.concatenate((sig_indicator_variables, decile_variables, continuous_variables), axis=None)

## 2. Run analyses
### a) Build model

Define function that will tune model

In [17]:
data_test = data.copy()
tr, te = data[data['calibrat'] == 1], data[data['calibrat'] == 0]

mod = mba263.logit_reg(tr['churndep'], tr[all_variables], 0.1)
tr['pred'] = mod.predict(tr[all_variables])
tr['churndep'] = np.where(tr['pred'] > 0.5, 1, 0)

te['pred'] = mod.predict(te[all_variables])
te['churndep'] = np.where(te['pred'] > 0.5, 1, 0)

# (te.loc[te['churndep'] == 1]['churn'].mean() / te['churn'].mean()) * 100
# tr[['pred', 'churn', 'churndep']].head()
te.groupby('churndep')['churn'].mean()


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6715440857498963
            Iterations: 774
            Function evaluations: 829
            Gradient evaluations: 774


churndep
0    0.014334
1    0.027762
Name: churn, dtype: float64

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

# Create a custom scoring function based on lift calculations
def exp_churn_scoring(test_set, predict, actual):
   test_set['pred'] = predict
   pred_rate = test_set.loc[test_set['pred'] == 1]['churn'].mean()
   actual_rate = np.mean(actual)
   return (pred_rate / actual_rate)*100
    
# A function to identify the best parameter(s) for a model tested using cross-validation
def tune_hyperparameters(train_dataset, predict, variables, threshold, param_grid, mod_type):
   # Set up the KFold object
   kf = KFold(n_splits=5, shuffle=True, random_state=0)
   best_score = -np.inf
   
   # Loop over the parameter grid for logistic regressions and neural networks
   if mod_type == 'logit_reg' or mod_type == 'neural_network':
      for a in param_grid['a']:
         scores = []
         
         # Loop over the cross-validation folds
         for train_index, test_index in kf.split(train_dataset):

            # Split the dataset
            x_train, x_test = train_dataset.iloc[train_index][variables], train_dataset.iloc[test_index][variables]
            y_train, y_test = train_dataset.iloc[train_index][predict], train_dataset.iloc[test_index][predict]

            # Fit and run the model
            if mod_type == 'logit_reg':
               model = mba263.logit_reg(y_train, x_train, a)
            elif mod_type == 'neural_network':
               model = mba263.neural_network(y_train, x_train, a)
               
            # Score results
            y_pred = np.where(model.predict(x_test) > threshold, 1, 0)
            scores.append(exp_churn_scoring(train_dataset.iloc[test_index], y_pred, y_test))

         # Compute the average score
         mean_score = np.mean(scores)

         # If the score is better, record the parameter
         if mean_score > best_score:
            best_score = mean_score
            best_param = {'a': a}
         
         # Reset scores
         scores.clear()

   # Return the best parameter
   return best_param

In [33]:
### A function to run a logistic regression, a logistic regression w/ regularization, 
### a neural network, and a random forest
### Function adds two columns, p for probability and res if selected for targeting
def run_ml_models(dataset, actual, variables, threshold):
    df = dataset.copy()

    # Split dataset into train and test based on whether calibrat = 0 or 1
    train, test = data[data['calibrat'] == 1], data[data['calibrat'] == 0]

    ### Use a logistic regression with regularization
    ### ------------------------------------------------
    logit_reg_param_grid = {'a': [0.01, 0.1, 0.5, 1, 3, 5]}
    best_param_logit_reg = tune_hyperparameters(
        train, actual, variables, threshold, logit_reg_param_grid, 'logit_reg')

    # Use the tuned parameter to predict
    result_logit_reg = mba263.logit_reg(train[actual], train[variables], best_param_logit_reg['a'])
    train['p_logr'] = result_logit_reg.predict(train[variables])
    train['res_logr'] = np.where(train['p_logr'] > threshold, 1, 0)
    test['p_logr'] = result_logit_reg.predict(test[variables])
    test['res_logr'] = np.where(test['p_logr'] > threshold, 1, 0)

    # Rename churn to actuals
    train['actual_res'] = train['churn']
    test['actual_res'] = test['churn']

    # Return three items, the results on the test dataset, the results on the training, and the models
    return [test[['customer', 'p_logr','actual_res', 'res_logr',]],
            train[['customer', 'p_logr','actual_res', 'res_logr',]], 
               [result_logit_reg],
               [best_param_logit_reg]]

Tune and run the logistic regression based on the dataset

In [34]:
predictions, actuals, model, params = run_ml_models(data, "churndep", all_variables, 0.5)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6721001181510932
            Iterations: 581
            Function evaluations: 635
            Gradient evaluations: 581
118.02124705350512
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6720374946167564
            Iterations: 615
            Function evaluations: 670
            Gradient evaluations: 615
116.65035424515754
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.671395750965719
            Iterations: 660
            Function evaluations: 716
            Gradient evaluations: 660
117.58896992921049
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6701912102622263
            Iterations: 551
            Function evaluations: 607
            Gradient evaluations: 551
114.64144554864595
Optimization terminated successfully    (Exit mode 0)
            Current

## 3. Evaluation
### a) Evaluate model characteristics
Print parameters, coefficients, and odds for the resulting model

In [37]:
from IPython.display import display
def print_params(params):
    print("Logistic Regression Regularization Parameter: {}".format(params[0]['a']))

# Print the coefficients for the logistic regressions
def print_coefs(models):
    print("Logistic Regression w/ Regularization Coefficients")
    display(models[0].summary())

# Print the odds ratios for each logistic regression
def print_odds(models):
    print("Logistic Regression w/ Regularization Odds Ratios")
    display(mba263.odds_ratios(models[0]))

print_params(params)
print_coefs(model)
print_odds(model)

Logistic Regression Regularization Parameter: 0.1
Logistic Regression w/ Regularization Coefficients


<class 'statsmodels.iolib.summary.Summary'>
"""
                        Mba263Logit Regression Results                        
==============================================================================
Dep. Variable:               churndep   No. Observations:                38941
Model:                    Mba263Logit   Df Residuals:                    38875
Method:                           MLE   Df Model:                           65
Date:                Mon, 01 Apr 2024   Pseudo R-squ.:                 0.03117
Time:                        10:52:51   Log-Likelihood:                -26150.
converged:                       True   LL-Null:                       -26992.
Covariance Type:            nonrobust   LLR p-value:                1.177e-308
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1487      0.095      1.561      0.119      -0.038       0.335
revenue        0.0020      0.001      2.460      0.014       0.000       0.004
mou           -0.0003   4.96e-05     -5.657      0.000      -0.000      -0.000
recchrge      -0.0031      0.001     -3.513      0.000      -0.005      -0.001
directas      -0.0012      0.006     -0.202      0.840      -0.013       0.010
overage        0.0008      0.000      2.712      0.007       0.000       0.001
roam           0.0071      0.002      3.437      0.001       0.003       0.011
changem       -0.0005   5.35e-05     -9.194      0.000      -0.001      -0.000
changer        0.0023      0.000      6.247      0.000       0.002       0.003
dropvce        0.0113      0.007      1.561      0.119      -0.003       0.026
blckvce        0.0064      0.007      0.893      0.372      -0.008       0.020
unansvce       0.0009      0.000      2.058      0.040    4.41e-05       0.002
custcare      -0.0060      0.003     -2.331      0.020      -0.011      -0.001
threeway      -0.0303      0.011     -2.690      0.007      -0.052      -0.008
mourec         0.0001      0.000      1.018      0.309      -0.000       0.000
outcalls       0.0011      0.001      1.894      0.058    -3.9e-05       0.002
incalls       -0.0031      0.001     -2.937      0.003      -0.005      -0.001
peakvce       -0.0007      0.000     -3.058      0.002      -0.001      -0.000
opeakvce      -0.0002      0.000     -0.782      0.434      -0.001       0.000
dropblk       -0.0031      0.007     -0.440      0.660      -0.017       0.011
callfwdv      -0.0026      0.023     -0.113      0.910      -0.048       0.043
callwait       0.0021      0.003      0.662      0.508      -0.004       0.008
months        -0.0213      0.002    -10.648      0.000      -0.025      -0.017
uniqsubs       0.1843      0.020      9.224      0.000       0.145       0.224
actvsubs      -0.2056      0.028     -7.368      0.000      -0.260      -0.151
phones         0.0487      0.018      2.679      0.007       0.013       0.084
models         0.0138      0.028      0.495      0.620      -0.041       0.068
eqpdays        0.0014   7.47e-05     19.309      0.000       0.001       0.002
age1          -0.0033      0.001     -3.784      0.000      -0.005      -0.002
age2          -0.0012      0.001     -1.718      0.086      -0.003       0.000
children       0.0945      0.028      3.359      0.001       0.039       0.150
credita       -0.1779      0.036     -5.010      0.000      -0.247      -0.108
creditaa      -0.3624      0.035    -10.482      0.000      -0.430      -0.295
prizmrur       0.0664      0.050      1.340      0.180      -0.031       0.164
prizmub       -0.0396      0.024     -1.622      0.105      -0.087       0.008
prizmtwn       0.0463      0.031      1.471      0.141      -0.015       0.108
refurb         0.2340      0.032      7.322      0.000       0.171       0.297
webcap        -0.1558      0.038     -4.149      0.000      -0.229      -0.082
tr

Logistic Regression w/ Regularization Odds Ratios


,Odds ratios,std err,z,P>|z|,[0.025,0.975]
revenue,1.001965,0.000800,2.457689,0.014,1.000414,1.003517
mou,0.999719,0.000050,5.658118,0.000,0.999623,0.999815
recchrge,0.996883,0.000886,3.518326,0.000,0.995164,0.998602
directas,0.998804,0.005931,0.201717,0.840,0.987296,1.010311
overage,1.000761,0.000281,2.710574,0.007,1.000216,1.001305
...,...,...,...,...,...,...
income,0.986875,0.005953,2.204618,0.027,0.975325,0.998425
mcycle,1.129112,0.100467,1.285118,0.199,0.934206,1.324018
setprcm,0.908459,0.036804,2.487230,0.013,0.837059,0.979860
setprc,1.000622,0.000283,2.198606,0.028,1.000073,1.001171


### b) Analyze Lift

In [39]:
print(predictions['actual_res'].mean())
print(predictions.loc[predictions['res_logr'] == 1]['actual_res'].mean())

0.019615421779882115
0.02776185406600606
